In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("test_session").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/13 20:20:30 WARN Utils: Your hostname, tatiane-Inspiron-3583, resolves to a loopback address: 127.0.1.1; using 192.168.1.52 instead (on interface wlo1)
25/10/13 20:20:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/13 20:20:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 41212)
Traceback (most recent call last):
  File "/usr/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.12/socketse

In [2]:
import sys, os, time, getpass

sys.path.append("/home/tatiane/lib/")

import pessoal
from pessoal import *
#print('AppID: ', sc.applicationId)

# Pedir ao spark para exibir só erros, escondendo avisos (WARN) e infos
# O correto é resolver a causa como, por exemplo: ordenados = Window.partitionBy().orderBy(dfs[i].columns[0])
spark.sparkContext.setLogLevel("ERROR")

Tempo inicial da execucao: 2025-10-13 20:20:41.809066
User: tatiane
Node: tatiane-Inspiron-3583


## Leitura

In [8]:
csv_path = "/home/tatiane/Downloads/base_dados_suja.csv"
df = spark.read.csv(csv_path, header=True, inferSchema=True)
pessoal.completudeSchema(df)

Qtd. registros: 100 | Quantidade de colunas:  5
root
 |-- nome: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- municipio_nascimento: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_unico: integer (nullable = true)



### Padronização de datas

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from datetime import datetime
import locale
from pyspark.sql import DataFrame

# Função Python para padronizar datas com formato flexível
def parse_date_str(date_str, output_format="DD/MM/YYYY"):
    if not date_str:
        return None
    date_str = str(date_str).strip()
    
    # Determina o formato de saída
    if output_format == "YYYY/MM/DD":
        fmt_out = "%Y/%m/%d"
    else:
        fmt_out = "%d/%m/%Y"
    
    # Tentar DD/MM/AAAA
    try:
        return datetime.strptime(date_str, "%d/%m/%Y").strftime(fmt_out)
    except:
        pass

    # Tentar AAAA/MM/DD
    try:
        return datetime.strptime(date_str, "%Y/%m/%d").strftime(fmt_out)
    except:
        pass

    # Tentar DD/mês/AAAA em português
    try:
        locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
        return datetime.strptime(date_str, "%d/%B/%Y").strftime(fmt_out)
    except:
        pass

    # Tentar DD/mês/AAAA em inglês
    try:
        locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
        return datetime.strptime(date_str, "%d/%B/%Y").strftime(fmt_out)
    except:
        pass

    return None

# Registrar como UDF do Spark com parâmetro de formato
def create_parse_date_udf(output_format="DD/MM/YYYY"):
    return F.udf(lambda x: parse_date_str(x, output_format), StringType())

In [11]:
# Função principal para aplicar em um DataFrame
def padroniza_datas(df: DataFrame, coluna: str, nova_coluna: str = None, output_format="DD/MM/YYYY") -> DataFrame:
    """
    Padroniza datas de uma coluna para o formato desejado.
    
    Args:
        df: DataFrame do PySpark
        coluna: nome da coluna com datas originais
        nova_coluna: nome da coluna de saída (default: coluna + '_padronizada')
        output_format: "DD/MM/YYYY" ou "YYYY/MM/DD"
        
    Returns:
        DataFrame com nova coluna de datas padronizadas
    """
    if nova_coluna is None:
        nova_coluna = coluna + "_padronizada"
    
    parse_udf = create_parse_date_udf(output_format)
    return df.withColumn(nova_coluna, parse_udf(F.col(coluna)))

In [12]:
df_padronizado = padroniza_datas(df, "data_nascimento", output_format="YYYY/MM/DD")

In [13]:
df_padronizado.select("data_nascimento", "data_nascimento_padronizada").show(10, truncate=False)

[Stage 6:>                                                          (0 + 1) / 1]

+-----------------+---------------------------+
|data_nascimento  |data_nascimento_padronizada|
+-----------------+---------------------------+
|15/fevereiro/1997|1997/02/15                 |
|2001/07/16       |2001/07/16                 |
|12/fevereiro/1973|1973/02/12                 |
|1990/06/15       |1990/06/15                 |
|08/junho/2010    |2010/06/08                 |
|08/junho/1976    |1976/06/08                 |
|20/11/2003       |2003/11/20                 |
|13/outubro/2008  |2008/10/13                 |
|07/01/1971       |1971/01/07                 |
|20/10/1997       |1997/10/20                 |
+-----------------+---------------------------+
only showing top 10 rows


### Limpeza de nomes

In [14]:
df_padronizado.printSchema()

root
 |-- nome: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- municipio_nascimento: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_unico: integer (nullable = true)
 |-- data_nascimento_padronizada: string (nullable = true)



In [15]:
df_padronizado.select('nome').show(50, truncate=False)

+-----------------------+
|nome                   |
+-----------------------+
|_ok Pedro Gomes        |
|Camila _ok Monteiro    |
|_ok Ricardo Teixeira   |
|Paula Azevedo NULL     |
|Fernanda Martins NULL  |
|Rafael Lima            |
|NULL Sofia Melo        |
|  André Pinto          |
|123 Bruno Santos       |
|@ Carla Nogueira       |
|Maria Silva temp       |
|temp Beatriz Costa     |
|Ricardo Teixeira       |
|teste Camila Monteiro  |
|Carla Nogueira         |
|### Maria Silva        |
|Paula sem dado Azevedo |
|Rafael @ Lima          |
|Aline Barbosa sem dado |
|João Souza             |
|Ricardo Teixeira       |
|XX Ricardo Teixeira    |
|Pedro Gomes            |
|Maria Silva            |
|Marcos Fernandes       |
|Paula sem dado Azevedo |
|Bruno Santos           |
|André Pinto            |
|Beatriz Costa          |
|Fernanda Martins temp  |
|Carlos Pereira XX      |
|NULL Aline Barbosa     |
|Fernanda Martins teste |
|Ricardo Teixeira 123   |
|André Pinto @          |
|Juliana Alm

In [16]:
# Os espaços não são apagados porque não tem pelo quê substituí-los.
lista_expressoes = (r'\s+', '###', '_ok', '@','XX','teste','123','sem dado', 'NULL', 'temp', '-')

df_padronizado.withColumn('nome_novo', F.regexp_replace('nome', '|'.join(lista_expressoes), ' ')).select('nome', 'nome_novo').show(100, False)

+-----------------------+------------------+
|nome                   |nome_novo         |
+-----------------------+------------------+
|_ok Pedro Gomes        |  Pedro Gomes     |
|Camila _ok Monteiro    |Camila   Monteiro |
|_ok Ricardo Teixeira   |  Ricardo Teixeira|
|Paula Azevedo NULL     |Paula Azevedo     |
|Fernanda Martins NULL  |Fernanda Martins  |
|Rafael Lima            |Rafael Lima       |
|NULL Sofia Melo        |  Sofia Melo      |
|  André Pinto          | André Pinto      |
|123 Bruno Santos       |  Bruno Santos    |
|@ Carla Nogueira       |  Carla Nogueira  |
|Maria Silva temp       |Maria Silva       |
|temp Beatriz Costa     |  Beatriz Costa   |
|Ricardo Teixeira       |Ricardo Teixeira  |
|teste Camila Monteiro  |  Camila Monteiro |
|Carla Nogueira         |Carla Nogueira    |
|### Maria Silva        |  Maria Silva     |
|Paula sem dado Azevedo |Paula   Azevedo   |
|Rafael @ Lima          |Rafael   Lima     |
|Aline Barbosa sem dado |Aline Barbosa     |
|João Souz

In [17]:
df_limpeza = df_padronizado.withColumn('nome_novo', F.regexp_replace('nome', '|'.join(lista_expressoes), ' '))

In [18]:
df_limpeza.select('nome', 'nome_novo').show(100, False)

+-----------------------+------------------+
|nome                   |nome_novo         |
+-----------------------+------------------+
|_ok Pedro Gomes        |  Pedro Gomes     |
|Camila _ok Monteiro    |Camila   Monteiro |
|_ok Ricardo Teixeira   |  Ricardo Teixeira|
|Paula Azevedo NULL     |Paula Azevedo     |
|Fernanda Martins NULL  |Fernanda Martins  |
|Rafael Lima            |Rafael Lima       |
|NULL Sofia Melo        |  Sofia Melo      |
|  André Pinto          | André Pinto      |
|123 Bruno Santos       |  Bruno Santos    |
|@ Carla Nogueira       |  Carla Nogueira  |
|Maria Silva temp       |Maria Silva       |
|temp Beatriz Costa     |  Beatriz Costa   |
|Ricardo Teixeira       |Ricardo Teixeira  |
|teste Camila Monteiro  |  Camila Monteiro |
|Carla Nogueira         |Carla Nogueira    |
|### Maria Silva        |  Maria Silva     |
|Paula sem dado Azevedo |Paula   Azevedo   |
|Rafael @ Lima          |Rafael   Lima     |
|Aline Barbosa sem dado |Aline Barbosa     |
|João Souz

In [19]:
df_limpeza = df_limpeza.withColumn("nome_novo", F.regexp_replace(F.col("nome_novo"), r'\s+', ' '))

In [20]:
df_limpeza = df_limpeza.withColumn("nome_limpo", F.trim(F.col("nome_novo")))

In [21]:
df_limpeza.select('nome', 'nome_novo', 'nome_limpo').show(100, False)

+-----------------------+-----------------+----------------+
|nome                   |nome_novo        |nome_limpo      |
+-----------------------+-----------------+----------------+
|_ok Pedro Gomes        | Pedro Gomes     |Pedro Gomes     |
|Camila _ok Monteiro    |Camila Monteiro  |Camila Monteiro |
|_ok Ricardo Teixeira   | Ricardo Teixeira|Ricardo Teixeira|
|Paula Azevedo NULL     |Paula Azevedo    |Paula Azevedo   |
|Fernanda Martins NULL  |Fernanda Martins |Fernanda Martins|
|Rafael Lima            |Rafael Lima      |Rafael Lima     |
|NULL Sofia Melo        | Sofia Melo      |Sofia Melo      |
|  André Pinto          | André Pinto     |André Pinto     |
|123 Bruno Santos       | Bruno Santos    |Bruno Santos    |
|@ Carla Nogueira       | Carla Nogueira  |Carla Nogueira  |
|Maria Silva temp       |Maria Silva      |Maria Silva     |
|temp Beatriz Costa     | Beatriz Costa   |Beatriz Costa   |
|Ricardo Teixeira       |Ricardo Teixeira |Ricardo Teixeira|
|teste Camila Monteiro  

In [22]:
df_limpeza = df_limpeza.drop('nome_novo', 'nome', 'data_nascimento')
df_limpeza = df_limpeza.withColumnRenamed('nome_limpo', 'nome')
df_limpeza = df_limpeza.withColumnRenamed('data_nascimento_padronizada', 'data_nascimento')

### Escrita da base

In [23]:
write_path = '/home/tatiane/Documentos/tratamento_dados/'

In [24]:
writeSingleCsv(df_limpeza, write_path)
# arquivo foi renomeado para: base_final.csv

('/home/tatiane/Documentos/tratamento_dados/tratamento_dados.csv',
 1.8179035329994804,
 1.8559890420001466)

### Finalização do notebook

In [25]:
executionTime()

Tempo de execucao ate este ponto: 0:00:19.171610
